# 1. Import libraries

In [1]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import src.geo_functions as gf
from bson.json_util import dumps
import json
import pandas as pd

# Importing the madrid functions:
import src.geo_functions as gf

# 2. Make MongoDB Connection

In [2]:
client = MongoClient("localhost:27017")
db = client.get_database("geospatial")

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'geospatial')

In [3]:
lisbon = db.get_collection("lisbon")
london = db.get_collection("london")
madrid = db.get_collection("madrid")

# 3. Indexes creation

In [ ]:
db.lisbon.create_index([("location", GEOSPHERE)])
db.london.create_index([("location", GEOSPHERE)])
db.madrid.create_index([("location", GEOSPHERE)])

# 4. Some extra cleanning

In [4]:
diccionario = {"Pub": "Bar",
            "Metro": 'Train Station',
            "Café": "Bar",
            "Bar": "Bar",
            "Comedy Club": "Bar",
            "Steakhouse": "Bar",
            "Hotel Bar": "Bar",
            "Cocktail":"Bar",
            "Lounge": "Bar",
            "Nightclub": "Bar",
            "Restaurant": "Bar",
            "Sports Bar": "Bar", 
            "Bistro": "Bar",
            "Wine Bar": "Bar",
            "Pizza": "Bar",  
            'Bus Stop': 'Train Station',
            "Nightclub": "Bar",
            "Spanish": 'Bar',
            "Winery": "Bar",
            "Cocktail": "Bar",
            "Tapas": "Bar",
            "Coffee Shop": "Starbucks", 
            "Japanese": "Bar",
            "Seafood": "Bar",  
            "Brewery":"Bar",
            "Diner": "Bar",
            "Breakfast": "Bar",
            "Chinese": "Bar",
            'New American': 'Bar',
            "Cafeteria": "Bar",
            "Salon / Barbershop": "Bar",
            "Roof Deck": "Bar",
            "Sushi": "Bar", 
            "Snacks": "Bar",  
            "Karaoke":"Bar",
            "Corporate Cafeteria": "Bar",
            "Ice Cream": "Bar",
            'Portuguese': 'Bar',
            'Train Station': 'Train Station'}

# 5. Geoqueries

## LONDON 

In [62]:
london_ = gf.geocode("St Paul´s Cathedral, London")

In [63]:
query_london = [{"$geoNear" : {'near': [51.51421, -0.05688],
                        'distanceField' : 'distance',
                        'maxDistance' : 1000,
                        'distanceMultiplier': 6371,
                        'spherical' : True}}]

geoloc_london = london.aggregate(query_london)
london_json = json.loads(dumps(geoloc_london))

In [ ]:
# Now we create a dataframe out of the json obtained:

In [64]:
london_df = pd.DataFrame(london_json)

In [11]:
gf.drop_columns(london_df)

In [12]:
london_df["city"] = "London"

In [13]:
london_df['shortname'] = london_df['shortname'].map(diccionario)

## LISBON

In [15]:
lisbon_ = gf.geocode("Avenida Antonio Augusto de Aguiar, Lisboa")

In [17]:
query_lisbon = [{"$geoNear" : {'near': [38.73280, -9.15221],
                        'distanceField' : 'distance',
                        'maxDistance' : 1000,
                        'distanceMultiplier': 6371,
                        'spherical' : True}}]

geoloc_lisbon = lisbon.aggregate(query_lisbon)
lisbon_json = json.loads(dumps(geoloc_lisbon))

In [19]:
# Now we create a dataframe out of the json obtained:

In [20]:
lisbon_df = pd.DataFrame(lisbon_json)

In [21]:
gf.drop_columns(lisbon_df)

In [22]:
lisbon_df["city"] = "Lisbon"

In [23]:
lisbon_df['shortname'] = lisbon_df['shortname'].map(diccionario)

## MADRID

In [25]:
madrid_ = gf.geocode("Paseo de la Castellana, Madrid")

In [27]:
query_madrid = [{"$geoNear" : {'near': [40.46879, -3.68915],
                        'distanceField' : 'distance',
                        'maxDistance' : 1000,
                        'distanceMultiplier': 6371,
                        'spherical' : True}}]

geoloc_madrid = madrid.aggregate(query_madrid)
madrid_json = json.loads(dumps(geoloc_madrid))

In [29]:
# Now we create a dataframe out of the json obtained:

In [30]:
madrid_df = pd.DataFrame(madrid_json)

In [31]:
gf.drop_columns(madrid_df)

In [32]:
madrid_df["city"] = "Madrid"

In [33]:
madrid_df['shortname'] = madrid_df['shortname'].map(diccionario)

# 6. DATA FOR PONDERATIONS

In [56]:
# First, we create a new dataframe with the information of the three cities:

In [48]:
data = madrid_df.append([lisbon_df, london_df])

In [57]:
# Second, we apply the ponderation funcions:

In [49]:
data["Puntuation"] = data["shortname"].apply(gf.puntuacion)

In [ ]:
# Third, we create a new column "Total":

In [50]:
data["Total"] = data["distance"]*data["Puntuation"]

In [ ]:
# Fourth, we create a new dataframe where we can see the result:

In [58]:
final_punctuations = pd.DataFrame(data.groupby(['city'])["Total"].sum())

In [59]:
final_punctuations

,Total
city,
Lisbon,1553.270196
London,1374.092475
Madrid,917.717745


As we can see in the latter table, Lisbon will be the best city to open the company, as it fulfils the most requirements.

In [61]:
gf.normalizacion(final_punctuations, "Total")

[1.0, 0.7180756359568398, 0.0]

# 7. Visualization

In [43]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [53]:
madrid_

{'type': 'Point', 'coordinates': ['40.46879', '-3.68915']}

In [54]:
map_1 = folium.Map(location=[40.46879,-3.68915], zoom_start=15)
map_1;

In [55]:
for i,row in data.iterrows():
    distrito = {"location": [row["latitud"], row["longitud"]], "tooltip": row["name"]}
    
    if row["shortname"] == "Starbucks":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="laptop",
                     icon_color="black"
        )
    elif row["shortname"] == "Train Station":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="glass",
                     icon_color="black")
        
    elif row["shortname"] == "Bar":
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="bug",
                     icon_color="black")
        
    mark = Marker(**distrito, icono=Icon)
    mark.add_to(map_1)
map_1;